In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

bits = np.array([1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1])

def conv_sp(bits, p):
    return np.reshape(bits, (-1, p))

table = {
    1: [1,1,1], 
    2: [1,1,0], 
    3: [0,1,0],
    4: [0,1,1],
    5: [0,0,1],
    6: [0,0,0],
    7: [1,0,0],
    8: [1,0,1]
}


# function to return key for any value 
def get_key(val): 
    for key, value in table.items(): 
         if np.array_equal(val,value): 
            return key
    return "key doesn't exist"

def s(i, E, M):
     return (np.sqrt(E)*np.cos(2*(i-1)*(np.pi/M)), np.sqrt(E)*np.sin(2*(i-1)*(np.pi/M)))

def freq_shift(s, fc, t):
    return (s[0]*np.sqrt(2/t[-1])*np.cos(2*np.pi*fc*t))+(s[1]*np.sqrt(2/t[-1])*np.sin(2*np.pi*fc*t))

E = 12
M =  8
fc = 1e4
Ts = 1e-3
p=3

conv = conv_sp(bits,p)
ts = np.linspace(0, Ts, 1000)
qpsk = np.array([])
for signal in conv:
    i = get_key(signal)
    qpsk = np.append(qpsk, freq_shift(s(i, E, M), fc, ts))

t = np.linspace(0, Ts*conv.shape[0], qpsk.size)
plt.plot(t, qpsk)
plt.ylabel("Amplitude (V)")
plt.xlabel("time (s)")
plt.show()





psi_1 = np.sqrt(2/Ts)*np.cos(2*np.pi*fc*t)
psi_2 = np.sqrt(2/Ts)*np.sin(2*np.pi*fc*t)
a1 = qpsk*psi_1
a2 = qpsk*psi_2

plt.figure()
plt.plot(t, a1,color='blue')
plt.ylabel("Amplitude (V)")
plt.xlabel("time (s)")

plt.plot(t, a2,color='red')
plt.ylabel("Amplitude (V)")
plt.xlabel("time (s)")


from scipy.integrate import simps

X = np.array([])
Y = np.array([])
for x in range(conv.shape[0]):
    X = np.append(X, simps(a1[ts.size*x:ts.size*(x+1)], ts))
    Y = np.append(Y, simps(a2[ts.size*x:ts.size*(x+1)], ts))
    

##calculo do arco tangente
phi_received = []
phi_received = (np.arctan2(Y,X)*180/np.pi + 360) % 360
print(phi_received) 

phi_constelar= []
for i in range(1,M+1):
  phi_constelar= np.append(phi_constelar,(2*(i-1)*(180/M)))
print(phi_constelar)

index = []
for i in phi_received:
    index.append(np.argmin(abs(i-phi_constelar)))

print(index)

bits_received=np.array([])

for x in index:
  bits_received = np.append(bits_received,table[x+1])

print(bits_received)
print(bits)





[1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1.]
[1 1 0 0 1 0 1 0 1 1 1 1]
